In [ ]:
from google.colab import drive; drive.mount('/content/drive')   # OK to enable, if your kaggle.json is stored in Google Drive

Mounted at /content/drive


In [ ]:
!pip -q install --upgrade --force-reinstall --no-deps kaggle > log  # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                                           # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
!cp kaggle.json ~/.kaggle/kaggle.json > log                   # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                              # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v 24apr23cml-crystals         # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log                          # download competition dataset as a zip file
!unzip -o *.zip >> log                                        # Kaggle dataset is copied as a single file and needs to be unzipped.
!kaggle competitions leaderboard --show                       # print public leaderboard

cp: cannot stat 'kaggle.json': No such file or directory
- competition is now set to: 24apr23cml-crystals
unzip:  cannot find or open *.zip, *.zip.zip or *.zip.ZIP.

No zipfiles found.
Using competition: 24apr23cml-crystals
403 - Forbidden


In [ ]:
%%time
%%capture
%reset -f
!pip -q install scikit-learn-extra > log
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all" 
from sklearn_extra.cluster import KMedoids
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import LabelEncoder
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, os

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

np.set_printoptions(linewidth=10000, precision=2, edgeitems=20, suppress=True)
pd.set_option('display.max_colwidth', 1000, 'display.max_columns', 100, 'display.width', 1000, 'display.max_rows', 4)

CPU times: user 1.43 s, sys: 221 ms, total: 1.65 s
Wall time: 13.3 s


`IsTest=1` are the observations of interest, but you can use others to create more reliable clusters. `Y` contains only 3 labels. You can deduce the fourth, `'orthorhombic'`.

In [ ]:
dfRaw = pd.read_csv('/content/drive/MyDrive/crystals_perovskites.csv'); dfRaw
#dfRaw = pd.read_csv('./crystals_perovskites.csv'); dfRaw

,IsTest,Y,v(A),v(B),r(A_XII)(Å),r(A_VI)(Å),r(B_VI)(Å),EN(A),EN(B),l(A-O)(Å),l(B-O)(Å),ΔENR,tG,τ,μ
0,1,cubic,1,5,1.28,0.95,0.61,1.93,2.16,2.761453,1.789300,-1.663536,0.942809,4.126280,0.435714
1,1,rhombohedral,1,5,1.28,0.95,0.76,1.93,2.02,2.761453,2.215655,-1.992714,0.877336,4.072914,0.542857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6002,0,NaN,0,0,0.89,0.72,0.74,1.33,1.65,2.383420,2.096141,-2.035750,0.756670,NaN,0.528571
6003,0,NaN,0,0,0.89,0.72,0.72,1.33,1.33,2.383420,2.043778,-2.097821,0.763809,NaN,0.514286


In [ ]:
tmr = Timer() # runtime limit (in seconds). Add all of your code after the timer

⏳ started. You have 60 sec. Good luck!


<hr color=red>

<font size=5>⏳</font> <strong><font color=orange size=5>Your Code, Documentation, Ideas and Timer - All Start Here...</font></strong>

Students: Keep all your definitions, code, documentation **between** ⏳ symbols.

## **Task 1. Preprocessing Pipeline**
 
Explain elements of your preprocessing pipeline i.e. feature engineering, subsampling, clustering, dimensionality reduction, etc. 
1. Why did you choose these elements? (Something in EDA, prior experience,...? Btw, EDA is not required)
1. How do you evaluate the effectiveness of these elements? 
1. What else have you tried that worked or didn't? 

**Student's answer:**

## **Task 2. Modeling Approach**
Explain your modeling approach, i.e. ideas you tried and why you thought they would be helpful. 

1. How did these decisions guide you in modeling?
1. How do you evaluate the effectiveness of these elements? 
1. What else have you tried that worked or didn't? 

**Student's answer:**

In [ ]:
df = dfRaw.query('IsTest==1').iloc[:,2:]
df = df.apply(lambda x: x.fillna(x.median())) # replace missing values with column mean
df = df.apply(LabelEncoder().fit_transform) # encode categorical features

In [ ]:
KTrue, K = 7, 7 
Cols = df.columns
for K in range(2, 30):
  km = KMedoids(n_clusters=K, random_state=0).fit(df[Cols])
  df['Cl'] = km.labels_
  print(f'Number of clusters: {K}')
  print('Medoid indices: ', km.medoid_indices_)
  print('Medoids: \n', km.cluster_centers_)

Number of clusters: 2
Medoid indices:  [505 379]
Medoids: 
 [[  1.   2.  23.  15.   8.  11.  41.  22.   0. 471. 299. 269.   8.]
 [  0.   3.  28.  19.  17.   3.  33.  30.  32. 164. 267. 261.  17.]]
Number of clusters: 3
Medoid indices:  [505 673  40]
Medoids: 
 [[  1.   2.  23.  15.   8.  11.  41.  22.   0. 471. 299. 269.   8.]
 [  1.   2.  14.  14.  13.   7.  13.  11.  19. 152.  95. 447.  13.]
 [  1.   2.  27.  11.  12.  29.  11.  10.  17. 185. 296. 234.  12.]]
Number of clusters: 4
Medoid indices:  [179 176 413 236]
Medoids: 
 [[  1.   2.  17.  17.   8.  13.  41.  15.   0. 525. 217. 368.   8.]
 [  1.   2.  17.  17.  17.  13.   5.  15.  38. 114.  98. 445.  17.]
 [  1.   2.  25.  16.  10.  10.  16.  23.   8. 159. 304. 245.  10.]
 [  2.   1.  12.  13.   4.  12.  25.  19.  10. 430. 195.  92.   4.]]
Number of clusters: 5
Medoid indices:  [ 82 599 413 236 176]
Medoids: 
 [[  1.   2.  24.  18.   4.   6.  38.  26.  16. 440. 341. 257.   4.]
 [  1.   2.  10.  15.   5.  16.  38.   6.  15. 552. 1

In [ ]:
dfLabeled = dfRaw.loc[~dfRaw.Y.isnull(),:]; dfLabeled  # We'll need these Y labels for naming clusters

,IsTest,Y,v(A),v(B),r(A_XII)(Å),r(A_VI)(Å),r(B_VI)(Å),EN(A),EN(B),l(A-O)(Å),l(B-O)(Å),ΔENR,tG,τ,μ
0,1,cubic,1,5,1.28,0.95,0.61,1.93,2.16,2.761453,1.789300,-1.663536,0.942809,4.126280,0.435714
1,1,rhombohedral,1,5,1.28,0.95,0.76,1.93,2.02,2.761453,2.215655,-1.992714,0.877336,4.072914,0.542857
2,1,tetragonal,2,4,1.61,1.35,0.97,0.89,2.10,2.942999,2.115492,-2.845607,0.898055,3.994771,0.692857


We compute a (symmetric) matrix of dot products to be used in naming our clusters.

In [ ]:
dfDot = pd.DataFrame(dfLabeled[Cols].values.astype(float)@km.cluster_centers_.T, index=dfLabeled.Y, columns=range(K))
dfDot.style.background_gradient(cmap='coolwarm', axis=None).set_precision(2)

<ipython-input-9-8003762b76d5>:2: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  dfDot.style.background_gradient(cmap='coolwarm', axis=None).set_precision(2)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28
Y,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
cubic,1521.44,1063.65,1239.18,1170.69,1040.82,1136.09,865.09,1184.79,1277.71,1135.99,839.34,1478.23,1022.13,1379.98,842.94,845.40,-233.90,1798.67,771.74,425.81,1330.76,26.80,1647.56,1478.03,124.37,1179.40,1201.13,1100.02,2114.50
rhombohedral,1424.93,851.20,1120.68,1060.49,875.79,1029.16,699.75,1083.16,1192.11,991.24,653.52,1417.42,895.59,1242.85,675.56,825.98,-450.27,1767.63,551.51,345.87,1188.69,-142.33,1534.03,1334.46,-13.15,1123.63,1016.12,885.05,2085.82
tetragonal,1199.43,327.57,872.30,836.55,493.00,821.12,366.16,928.80,1051.64,689.32,256.65,1314.87,662.24,933.65,336.00,745.71,-951.32,1732.24,47.02,178.31,878.09,-557.49,1271.58,1097.46,-305.16,1047.81,607.01,382.10,1990.63


Naming clusters: We take the label of the observation closest to the medoid and name that cluster with the structure of the observation. Then repeat for the remaining clusters. The unnamed observations are labeled with `'orthorhombic'`.

In [ ]:
for _ in range(len(dfDot)):
  r, c = [i[0] for i in np.where(dfDot==dfDot.max().max())] # row/column of the largest dot product
  TrueLbl, ClLbl = dfDot.index[r], dfDot.columns[c]         # true labels are row indices, numeric cluster labels are column indices
  df.loc[df.Cl==ClLbl, 'structure'] = TrueLbl                      # swap numeric labels for true structure labels
  dfDot = dfDot.drop(TrueLbl, axis=0).drop(ClLbl, axis=1)   # drop used row and column
  # print(r, c, dfDot)

In [ ]:
df.loc[df['structure'].isnull(), 'structure'] = 'orthorhombic'  # rename observations in the remaining cluster
df['structure'].value_counts()   # show distribution of labels
df['structure'].to_csv('result7.csv', index_label='id')

orthorhombic    587
cubic            55
tetragonal       19
rhombohedral     14
Name: structure, dtype: int64

# **References:**

1. Remember to cite your sources here as well! At the least, your textbook should be cited. Google Scholar allows you to effortlessly copy/paste an APA citation format for books and publications. Also cite StackOverflow, package documentation, and other meaningful internet resources to help your peers learn from these (and to avoid plagiarism claims).
1. ...
1. ...

<font size=5>⏳</font> <strong><font color=orange size=5>Do not exceed competition's runtime limit!</font></strong>


In [ ]:
tmr.ShowTime()    # measure Colab's runtime. Do not remove. Keep as the last cell in your notebook.

Runtime is 4 sec


# 💡**Starter Ideas**

1. Try different hyperparameters and models
1. Try engineering new features, scaling, interactions, ...
  1. Look for linear and non-linear relations between features and target
1. Learn about [perovskite](https://en.wikipedia.org/wiki/Perovskite) crystals to build new features based on exact equations [&#127910;](https://www.youtube.com/results?search_query=perovskite+crystals). See [Santosh Behara's paper](https://www.sciencedirect.com/science/article/pii/S0927025620306820) for their feature engineering; they used supervised ML. The paper is saved to our shared Google Drive folder. Paper references can also help.
1. You don't have to use the 3 provided labels. 
1. [Silhouette](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html) and [other](https://scikit-learn.org/stable/modules/classes.html#clustering-metrics) metrics can help auto-identify stable clusters. 
1. If your model's performance is worse than random, consider auto-labeling your clusters differently. 
  1. E.g.: if a label map $[0,1]\to[A,B]$ in a balanced dataset is 40% accurate (i.e. worse than 50% from a random labeling), then the map $[0,1]\to[B,A]$ yields 60% accuracy.
1. Visualize your clusters in 2D using [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html), [tSNE](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html) or [UMAP](https://pypi.org/project/umap-learn/). 
  1. Do clusters separate well (i.e. tight within and far without)? Are cluster centers representative of their clusters? 
  1. FYI: [plotly](https://plotly.com/python/) is convenient in displaying more information for every point on hover, zooming, etc.
1. Cluster centers (especially centroids) can be sensitive to "outliers". Consider locating anamolic observations and limiting their effect on clusters.
 
